In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark._jvm.decoders.HRTDCDecoder.registerUDF(spark._jsparkSession)
spark._jvm.decoders.HRTDCUnpairedDecoder.registerUDF(spark._jsparkSession)

df = spark.read.parquet("/home/h487/data/parquet/run1081.parquet")
df.show(10)

# Filter SRPPAC strip data and decode
df_src = df.filter("femType==7 and femId==615").select("data").withColumn("decoded",F.expr("decode_hrtdc_unpaired_segdata(data)"))
df_src.show(10,truncate=False)
df_src = df_src.select("decoded.*").select("hbf.*","data").select("hbfNumber","data").filter("array_size(decoded.data)>1")
df_src.show(10,truncate=False)
df_src = df_src.withColumn("ex",F.explode("data")).select("hbfNumber","ex.*").withColumnRenamed("tot","edge")
#df_src.filter("hbfNumber=10057285 and ch=41").show()
df_src.show(5)

# Filter SRPPAC anode data and decode
df_sra = df.filter("femType==5 and femId==614").select("data").withColumn("decoded",F.expr("decode_hrtdc_segdata(data)"))
df_sra = df_sra.select("decoded.*").select("hbf.*","data").filter("array_size(decoded.data)>0")#.select("hbfNumber","data").filter("array_size(decoded.data)>0")
df_sra.show(5)
df_sra = df_sra.withColumn("ex",F.explode("data")).select("hbfNumber","ex.*").filter("ch==4")
df_sra = df_sra.groupBy("hbfNumber").agg(F.collect_list("time").alias("time"),F.collect_list("tot").alias("tot")) # select fastest anode hit
df_sra = df_sra.withColumn("size", F.size("time"))
df_sra.show(5)

#check = spark.read.parquet("/home/h487/data/parquet/run1039_srppac.parquet")
#check.show(20)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/16 06:05:08 WARN Utils: Your hostname, gpuana02, resolves to a loopback address: 127.0.1.1; using 192.168.1.216 instead (on interface enp4s0)
26/01/16 06:05:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
26/01/16 06:05:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/16 06:05:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/16 06:05:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
26/01/16 06:05:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
26/01/16 06:05:09 WARN Utils: Se

+----+-------------+----------+----+-------+--------+-------+-------+-----+-----------+----------+--------+--------+--------------------+
| run|time_frame_id|num_source|type|version|   magic|   tfId|femType|femId|numMessages|   timeSec|timeUSec|subMagic|                data|
+----+-------------+----------+----+-------+--------+-------+-------+-----+-----------+----------+--------+--------+--------------------+
|1081|            6|   3267100|   2|     83|SUBTIME |3267100|      7|  615|          2|1768509805|  502007|HRTBEAT |[16 91 DF 13 00 0...|
|1081|            6|   3267100|   2|     83|SUBTIME |3267100|      5|  617|          2|1768509805|  502347|HRTBEAT |[0B 8F DF 73 F6 2...|
|1081|            6|   3267100|   2|     83|SUBTIME |3267100|      5|  616|          2|1768509805|  502603|HRTBEAT |[A0 CA DF 73 72 2...|
|1081|            6|   3267100|   2|     83|SUBTIME |3267100|      5|  618|          2|1768509805|  502617|HRTBEAT |[1C DA 31 00 00 0...|
|1081|            6|   3267100|   

+---------+--------------------+--------------------+----+
|hbfNumber|                time|                 tot|size|
+---------+--------------------+--------------------+----+
|  2788146|[325429925, 52009...|      [83481, 83529]|   2|
|  2788152|[2714277, 3539305...|[83569, 83548, 83...|  19|
|  2788273|         [494636979]|             [83439]|   1|
|  2788305|[125741984, 13086...|      [83581, 83473]|   2|
|  2788329|         [289029715]|             [83626]|   1|
+---------+--------------------+--------------------+----+
only showing top 5 rows
